In [120]:
import datetime as dt
import ccxt

API_KEY = "gjfggEtlnUr0WyMNwViApfABmKYMsnqm3o34QV6iNTWYKV1xVfHn16XUNpoaCYhG"
API_SECRET = "On6NkNTmf5sPeEpVtIrp2B8uBburwhmJLbOjJZaEh3hc1nuAjPwqMxTPOAb7OSkm"

In [216]:
print('CCXT Version:', ccxt.__version__)

exchange = ccxt.binance({
    'enableRateLimit': True,
    "apiKey": API_KEY,
    "secret": API_SECRET,
})


## you can add a fourth parameter which is the endtime

def getActualPrice(symbol):
    now = str(dt.datetime.now())[:-7]
    timestamp = int(dt.datetime.strptime(now, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    response = exchange.fetch_ohlcv(symbol, '1m',timestamp-70000) # Gets Current Price 
    print(now,response[0][1],symbol)

getActualPrice('BTC/USDT')



CCXT Version: 1.72.29
2022-02-05 19:17:27 41723.45 BTC/USDT


In [17]:
# Convert epoch to realtime
print("EPOCH:",response[len(response)-1][0]/1000)
print("CONVERTED:",dt.datetime.utcfromtimestamp(response[len(response)-1][0]/1000).strftime('%Y-%m-%d %H:%M:%S'))

NameError: name 'response' is not defined

In [ ]:
prices = []
for symbol in exchange.markets:
    if "USDT" in symbol:
        response = exchange.fetch_ohlcv(symbol, '1m', timestamp,1) 
        if len(response) != 0:
            
            prices.append([symbol,dt.datetime.utcfromtimestamp(response[len(response)-1][0]/1000).strftime('%Y-%m-%d %H:%M:%S'),response[0][1]])
            print(symbol,dt.datetime.utcfromtimestamp(response[len(response)-1][0]/1000).strftime('%Y-%m-%d %H:%M:%S'),response[0][1])
             # rate limit

In [ ]:
# Get order book
import time
delay = 2 # seconds
for symbol in exchange.markets:
    pprint (exchange.fetch_order_book (symbol))
    time.sleep (delay) # rate limit

In [161]:
# Get all symbols
symbols = []
for symbol in exchange.markets:
    if "/USDT" in symbol:
        symbols.append(symbol)
len(symbols)

386

In [112]:
len(exchange.fetch_order_book('BTC/USDT')['asks'])

100

In [ ]:
import nest_asyncio
nest_asyncio.apply()
__import__('IPython').embed()

In [54]:
import asyncio
import websockets
import json
import pandas as pd
import datetime as dt
import time

async def call_api(msg):
   async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
       await websocket.send(msg)
       while websocket.open:
           response = await websocket.recv()
           return response

def async_loop(api, message):
    return asyncio.get_event_loop().run_until_complete(api(message))


def retrieve_historic_data(start, end, instrument, timeframe):
    msg = \
        {
            "jsonrpc": "2.0",
            "id": 833,
            "method": "public/get_tradingview_chart_data",
            "params": {
                "instrument_name": instrument,
                "start_timestamp": start,
                "end_timestamp": end,
                "resolution": timeframe
            }
        }
    resp = async_loop(call_api, json.dumps(msg))

    return resp

def getInstruments (currency = 'BTC', type = 'option'):
    msg = \
    {
        "jsonrpc" : "2.0",
        "id" : 7617,
        "method" : "public/get_instruments",
        "params" : {
            "currency" : currency,
            "kind" : type,
            "expired" : False
        }
    }
    resp = async_loop(call_api, json.dumps(msg))

    return resp

def json_to_dataframe(json_resp):
    res = json.loads(json_resp)

    df = pd.DataFrame(res['result'])

    df['ticks'] = df.ticks / 1000
    df['timestamp'] = [dt.datetime.fromtimestamp(date) for date in df.ticks]

    return df




if __name__ == '__main__':
    start = 1604693130000
    end = round(time.time() * 1000)
    
    instrument = "BTC-PERPETUAL"
    timeframe = '1'

    #json_resp = retrieve_historic_data(start, end, instrument, timeframe)

    #df = json_to_dataframe(json_resp)
    #print(df.tail())

    json_resp = getInstruments()
    res = json.loads(json_resp)
    df = pd.DataFrame(res['result']).sort_values(["expiration_timestamp","instrument_name"])
    
    print(df.tail())




     tick_size  taker_commission   strike settlement_period quote_currency  \
16      0.0005            0.0003  50000.0             month            BTC   
236     0.0005            0.0003  55000.0             month            BTC   
169     0.0005            0.0003  55000.0             month            BTC   
94      0.0005            0.0003  60000.0             month            BTC   
29      0.0005            0.0003  60000.0             month            BTC   

    option_type  min_trade_amount  maker_commission    kind  is_active  \
16          put               0.1            0.0003  option       True   
236        call               0.1            0.0003  option       True   
169         put               0.1            0.0003  option       True   
94         call               0.1            0.0003  option       True   
29          put               0.1            0.0003  option       True   

         instrument_name  expiration_timestamp  creation_timestamp  \
16   BTC-30DEC22

In [52]:
time.strftime('%d-%m-%Y %H:%M:%S',time.localtime(1646380800000/1000))

'04-03-2022 05:00:00'